In [1]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import StratifiedKFold

%matplotlib inline
plt.rcParams['figure.figsize'] =  (17.0, 4.0) #Default (6.0, 4.0)

In [2]:
# TODO
# (0) Select the best [N, 2N] data points. At least N and then remaining N if they are above the FPR_TARGET level
# (1) Incorporate into the general framework
# (2) Combine model predictions for ensemble / voting framework
# (3) Personal models

In [3]:
# ----------------------------------------------------
# Get data
# ----------------------------------------------------
np.random.seed(15)
X, y = make_classification(n_samples=10000, n_features=30, n_informative=20) #80000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # test is only to be used at the very end!

In [4]:
# ----------------------------------------------------
# User defined
# ----------------------------------------------------
FPR_TARGET = 0.02
N_FOLDS = 5
N_PLAYERS = 17

In [5]:
# ----------------------------------------------------
# Result objects
# ----------------------------------------------------
roc_results = {} # NAME: [FPR-stats, TPR-stats]
fpr_results = {} # NAME: [matrix of FPR-stats]
thresh_min_results = {} # NAME: thresh_min
fpr_lb_results = {} # NAME: fpr_lb array

In [6]:
# ----------------------------------------------------
# Helper functions
# ----------------------------------------------------

# Interpolate FPR and TPR over thresh_standard
def roc_curve_interp(y_test_in, y_pred_in, thresh_standard):
    fpr, tpr, thresh = roc_curve(y_test_in, y_pred_in)
    fpr_interp = np.interp(thresh_standard, thresh, fpr, period=360)
    tpr_interp = np.interp(thresh_standard, thresh, tpr, period=360)
    return fpr_interp, tpr_interp

# Add FPR and TPR to result dicts
def add_to_results(fpr, tpr, name, roc_results, fpr_results):
    roc_results[name] = [fpr, tpr]
    if not(name in fpr_results.keys()): fpr_results[name] = fpr
    else: fpr_results[name] = np.vstack([fpr_results[name], fpr])
    return roc_results, fpr_results

# Get the minimum threshold for a particular classifier that we need in order to reach the FPR_TARGET
def get_thresh_min(name, fpr_results, thresh_standard):
    fpr_mean = np.mean(fpr_results.get(name), axis=0)
    fpr_var  = np.std(fpr_results.get(name), axis=0)
    fpr_lb   = fpr_mean + 1.96*fpr_var                     # fpr lower bound CI
    thresh_min = min(thresh_standard[fpr_lb < FPR_TARGET]) # minimum threshold required to achieve FPR_TARGET
    return [thresh_min, fpr_lb]

# Indexes of N max elements of a list
def get_N_max_idx(arr, N):
    return arr.argsort()[-N:][::-1]

# Closest index to val in arr
def closest_index(arr, val):
    return min(range(len(arr)), key=lambda i: abs(arr[i]-val))


In [7]:
# ----------------------------------------------------
# Classification models (not interpretable)
# ----------------------------------------------------
def classification_models(y_train_cv, X_train_cv, y_test_cv, X_test_cv, 
                          roc_results, fpr_results, thresh_standard, TORECORD):
    # for random classifiers
    n_estimator = 11 # TODO: can be tuned using CV. When we want to optimize
    
    # for results
    y_pred_results = {}

    # It is important to train the ensemble of trees on a different subset
    # of the training data than the linear regression model to avoid
    # overfitting, in particular if the total number of leaves is
    # similar to the number of training samples
    X_train_cv_small, X_train_lr, y_train_cv_small, y_train_lr = train_test_split(X_train_cv,
                                                                                  y_train_cv,
                                                                                  test_size=0.5)

    # Unsupervised transformation based on totally random trees
    rt = RandomTreesEmbedding(max_depth=3, n_estimators=n_estimator,random_state=0)

    rt_lm = LogisticRegression()
    pipeline = make_pipeline(rt, rt_lm)
    pipeline.fit(X_train_cv_small, y_train_cv_small)
    y_pred_rt = pipeline.predict_proba(X_test_cv)[:, 1]
    y_pred_results['RT + LR'] = y_pred_rt
    if TORECORD:
        fpr_rt_lm, tpr_rt_lm = roc_curve_interp(y_test_cv, y_pred_rt, thresh_standard)
        roc_results, fpr_results = add_to_results(fpr_rt_lm, tpr_rt_lm, 'RT + LR', roc_results, fpr_results)

    # Supervised transformation based on random forests
    rf = RandomForestClassifier(max_depth=3, n_estimators=n_estimator)
    rf_enc = OneHotEncoder()
    rf_lm = LogisticRegression()
    rf.fit(X_train_cv_small, y_train_cv_small)
    rf_enc.fit(rf.apply(X_train_cv_small))
    rf_lm.fit(rf_enc.transform(rf.apply(X_train_lr)), y_train_lr)

    y_pred_rf_lm = rf_lm.predict_proba(rf_enc.transform(rf.apply(X_test_cv)))[:, 1]
    y_pred_results['RF + LR'] = y_pred_rf_lm
    if TORECORD:
        fpr_rf_lm, tpr_rf_lm = roc_curve_interp(y_test_cv, y_pred_rf_lm, thresh_standard)
        roc_results, fpr_results = add_to_results(fpr_rf_lm, tpr_rf_lm, 'RF + LR', roc_results, fpr_results)


    # Supervised transformation based on gradient boosting
    grd = GradientBoostingClassifier(n_estimators=n_estimator)
    grd_enc = OneHotEncoder()
    grd_lm = LogisticRegression()
    grd.fit(X_train_cv_small, y_train_cv_small)
    grd_enc.fit(grd.apply(X_train_cv_small)[:, :, 0])
    grd_lm.fit(grd_enc.transform(grd.apply(X_train_lr)[:, :, 0]), y_train_lr)

    y_pred_grd_lm = grd_lm.predict_proba(
        grd_enc.transform(grd.apply(X_test_cv)[:, :, 0]))[:, 1]
    y_pred_results['GBT + LR'] = y_pred_grd_lm
    if TORECORD:
        fpr_grd_lm, tpr_grd_lm = roc_curve_interp(y_test_cv, y_pred_grd_lm, thresh_standard)
        roc_results, fpr_results = add_to_results(fpr_grd_lm, tpr_grd_lm, 'GBT + LR', roc_results, fpr_results)

    # The gradient boosted model by itself
    y_pred_grd = grd.predict_proba(X_test_cv)[:, 1]
    y_pred_results['GBT'] = y_pred_grd
    if TORECORD:
        fpr_grd, tpr_grd = roc_curve_interp(y_test_cv, y_pred_grd, thresh_standard)
        roc_results, fpr_results = add_to_results(fpr_grd, tpr_grd, 'GBT', roc_results, fpr_results)

    # The random forest model by itself
    y_pred_rf = rf.predict_proba(X_test_cv)[:, 1]
    y_pred_results['RF'] = y_pred_rf
    if TORECORD:
        fpr_rf, tpr_rf = roc_curve_interp(y_test_cv, y_pred_rf, thresh_standard)
        roc_results, fpr_results = add_to_results(fpr_rf, tpr_rf, 'RF', roc_results, fpr_results)
        
    return roc_results, fpr_results, y_pred_results

In [8]:
# ----------------------------------------------------
# Learn thresholds for each algorithm with y_train and X_train
# ----------------------------------------------------

# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(y_train, n_folds=N_FOLDS)

# Standard intervals for probability threshold
thresh_standard = np.linspace(1.0, 0.0, 1001)

# 
for idx_cv, (idx_train, idx_test) in enumerate(cv):
    y_train_cv = y_train[idx_train]
    X_train_cv = X_train[idx_train]
    y_test_cv  = y_train[idx_test]
    X_test_cv  = X_train[idx_test]
    
    roc_results, fpr_results, _ = classification_models(y_train_cv, X_train_cv, y_test_cv, X_test_cv, 
                                                      roc_results, fpr_results, thresh_standard, True)

# Get thresholds for each algorithm
for key in fpr_results.keys():
    thresh_min_results[key], fpr_lb_results[key] = get_thresh_min(key, fpr_results, thresh_standard)


In [9]:
# Simple emsemble that takes the product of the LB probabilites
def simple_ensemble(y_preds, fpr_lb_results):
    #init
    n_pred = len(y_preds.get(y_preds.keys()[0]))
    emsemble_pred = np.ones(n_pred)
    # P = \prod LB_i
    for key in thresh_min_results.keys():
        pred_curr = y_preds.get(key)
        fpr_curr  = fpr_lb_results.get(key)
        for ii in range(0,n_pred):
            emsemble_pred[ii] *= (1-fpr_curr[closest_index(thresh_standard, pred_curr[ii])])
    # store
    y_preds['Simple Emsemble'] = emsemble_pred
    return y_preds

In [10]:
# ----------------------------------------------------
# Save models
# ----------------------------------------------------

# TODO


In [11]:
# ----------------------------------------------------
# Test models to predict 1's correctly. 
# Objective is to obtain the FPR_TARGET in the test data.
# We also still want to actually make some predictions.
# ----------------------------------------------------

# Train with all training data and predict on testing data
_, _, y_preds = classification_models(y_train, X_train, y_test, X_test, 
                                                      [], [], thresh_standard, False)
# Show good prediction accuracy
for key in thresh_min_results.keys():
    preds = y_test[y_preds.get(key) > thresh_min_results.get(key)]
    print key
    if len(preds) == 0: print 'no predictions :('
    else: print 'Accuracy:', np.mean(preds), 'Number: ', len(preds)         
print '\n'



# Emsemble of classifiers
y_preds = simple_ensemble(y_preds, fpr_lb_results)

# Print accuracy of N_PLAYERS most probable (can also extract indexes from here)
print 'Picking the',N_PLAYERS,'most probable players'
for key in y_preds.keys():
    max_idx = get_N_max_idx(y_preds.get(key), N_PLAYERS)
    preds = y_test[max_idx]
    print key
    if len(preds) == 0: print 'no predictions :('
    else: print 'Accuracy:', np.mean(preds)


RF + LR
Accuracy: 0.995412844037 Number:  218
GBT + LR
Accuracy: 0.974358974359 Number:  234
RT + LR
Accuracy: 0.884615384615 Number:  52
RF
Accuracy: 0.984615384615 Number:  130
GBT
Accuracy: 0.992537313433 Number:  134


Picking the 17 most probable players
GBT
Accuracy: 1.0
RF + LR
Accuracy: 1.0
RF
Accuracy: 1.0
GBT + LR
Accuracy: 1.0
RT + LR
Accuracy: 0.882352941176
Simple Emsemble
Accuracy: 1.0


In [ ]:
# ----------------------------------------------------
# Plot ROC for 1 CV
# ----------------------------------------------------

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
for key in roc_results.keys():
    plt.plot(roc_results.get(key)[0], roc_results.get(key)[1], label=key)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()


plt.figure(2)
plt.xlim(0, 0.1)
plt.ylim(0, 1)
plt.plot([0, 1], [0, 1], 'k--')
for key in roc_results.keys():
    plt.plot(roc_results.get(key)[0], roc_results.get(key)[1], label=key)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()